# Fashion Mnist DNN Tutorial [CNN & Multi-layer Perceptron (MLP)]

Efficient Network
- https://github.com/lukemelas/EfficientNet-PyTorch
- image 시리즈 중 auto ML을 통해 효율적으로 구현한 것

외부 파일 가져오기 & requirements 설치

In [ ]:
!pwd

# mount
from google.colab import drive
drive.mount('/content/drive')

import os
import sys

drive_project_root = '/content/drive/MyDrive/#fastcampus'
sys.path.append(drive_project_root)

!ls

!pip install -r '/content/drive/MyDrive/#fastcampus/requirements.txt'

In [ ]:
pip install torch-optimizer

wandb 오류 있을 때 : `wandb.flush`

In [ ]:
pip install wandb

In [ ]:
pip install omegaconf

In [ ]:
pip install efficientnet_pytorch

https://pytorch.org/vision/stable/transforms.html

★ OmegaConf
- https://omegaconf.readthedocs.io/en/2.1_branch/
- hyperparameter configuration을 관리하기 위한 open source library
- DictConfig : Dictionary 형태의 configuration
- Hydra도 omegaconf를 기반으로 만들어짐
  - Hydra는 무겁기 때문에 omegaconf를 먼저 거치고, hydra 사용

gpu 확인

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

In [ ]:
from datetime import datetime

import numpy as np
from tqdm import tqdm   # 진행율
import matplotlib.pyplot as plt

from omegaconf import OmegaConf, DictConfig

import torch
from torch import nn
import torch.nn.functional as F     # relu 등 함수 모음
from torch import optim
from torch_optimizer import RAdam, AdamP
from torch.utils.tensorboard import SummaryWriter

from torchvision.datasets import FashionMNIST
from torchvision import transforms    # feature engineering 전처리 작업 효율적으로 할 수 있게 도와줌
from torch.utils.data import random_split  # dataset split

import wandb

from efficientnet_pytorch import EfficientNet

왼쪽 파일 부분 보면 'data' 라는 폴더가 새로 생겼고 FahionMNIST data download 된 것 볼 수 있음</br>
'raw' 파일 안에 압축 파일들 생성됨<br>
https://pytorch.org/vision/stable/generated/torchvision.datasets.FashionMNIST.html
- 링크 들어가서 보면 getitem으로 정의되어 있는 데이터라는 것 알 수 있음
- 이런 dataset은 iterator가 아니기 때문에 list 같이 index를 통해 특정 value 바로 접근 가능
- iterator dataset은 next 함수 이용해서 접근 가능

In [ ]:
data_root = os.path.join(os.getcwd(), 'data')

# 전처리 부분 (preprocessing) & 데이터 셀 출력
# transform = transforms.Compose(
#     [
#      transforms.ToTensor(),
#      transforms.Normalize([0.5], [0.5]), # mean, std
#     ]
# )

# This 'transform' is for EfficientNet
transform = transforms.Compose(
    [
     transforms.Resize(224),
     transforms.ToTensor(),
     # 원래 input 이미지 channel이 지금 1개(회색)인데 3개(color)로 확대 필요
     transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # efficientNet에 맞는 normalize 방식 (검색)
    ]
)

fashion_mnist_dataset = FashionMNIST(data_root, download=True, train=True, transform=transform)

In [ ]:
# list처럼 특정 이미지 바로 접근하기
# normalize해서 다운받았기 때문에 어느 정도 normalize된 데이터
fashion_mnist_dataset[0][0]

# 첫 번째 데이터의 label
fashion_mnist_dataset[0][1]

# dataset split
dset = random_split(fashion_mnist_dataset, [int(len(fashion_mnist_dataset)*0.7), int(len(fashion_mnist_dataset)*0.3)])

dset[0]

data_utils : 강사가 미리 만들어놓은 코드

In [ ]:
from data_utils import dataset_split

In [ ]:
datasets = dataset_split(fashion_mnist_dataset, split=[0.9, 0.1])
print(datasets)

dataloader 정의

In [ ]:
train_dataset = datasets["train"]
val_dataset = datasets["val"]

# batch 단위로 데이터를 묶을 예정
train_batch_size = 100
val_batch_size = 10

# num_workers : 병렬 processing
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=1
)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_batch_size, shuffle=False, num_workers=1
)

In [ ]:
for sample_batch in train_dataloader:
    print(sample_batch)
    # 그림과 label 각각에 대한 shape 확인
    print(sample_batch[0].shape, sample_batch[1].shape)
    break

## 모델 정의
- MLP (Multi-layer Perceptron)
- MLPWithDropout
- CNN
  - https://pytorch.org/docs/stable/nn.html#convolution-layers
    - conv2d : https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d
  - https://pytorch.org/docs/stable/nn.html#pooling-layers
    - maxpool2d : https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#torch.nn.MaxPool2d

pytorch model은 보통 class로 정의
- nn과 같은 module을 꼭 import 해야 함 ★ : GPU, CPU 간 꼬이는 것 방지

In [ ]:
# v1 : basic
class MLP(nn.Module):
    # input, hidden layer1/2, output의 차원을 먼저 알아야 함
    def __init__(self, in_dim: int, h1_dim: int, h2_dim: int, out_dim: int):
        # nn.Module 가져오기
        super().__init__()
        # MLP layer를 각각 정의할 때 제일 처음 linear layer를 지나야 함
        self.linear1 = nn.Linear(in_dim, h1_dim)
        self.linear2 = nn.Linear(h1_dim, h2_dim)
        self.linear3 = nn.Linear(h2_dim, out_dim)
        self.relu = F.relu
    
    def forward(self, input):
        # 위에서 input shape : torch.Size([100, 1, 28, 28]) → 1×28×28
        x = torch.flatten(input, start_dim=1)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        out = self.linear3(x)
        # output에 sigmoid(binary)나 softmax(multi) 씌워서 출력해도 되고 출력 후 나중에 작업해도 됨
        # out = F.softmax(out)
        return out

# v2 : regularizers (dropout, early stoping)
class MLPWithDropout(MLP):
    def __init__(self, in_dim: int, h1_dim: int, h2_dim: int, out_dim: int, dropout_prob: float):
        super().__init__(in_dim, h1_dim, h2_dim, out_dim)
        # Dropout에 2d, 3d 등도 있는데 linear에 적용할 땐 1d 사용
        self.dropout1 = nn.Dropout(dropout_prob)
        self.dropout2 = nn.Dropout(dropout_prob)

    def forward(self, input):
        x = torch.flatten(input, start_dim=1)
        x = self.relu(self.linear1(x))
        x = self.dropout1(x)
        x = self.relu(self.linear2(x))
        x = self.dropout2(x)
        out = self.linear3(x)
        return out

CNN

In [ ]:
# cnn hyperparameter configuration
_cnn_cfg_dict: dict = {
    # layer 하나에 convolution, maxpooling 모두 만들 예정
    'layer_1': {
        # conv2d 사이트 보면 in_channels, out_channels, kernel_size는 반드시 정의해야 됨
        'conv2d_in_channels': 1, # 흑백 이미지니까 input channel : 1
        'conv2d_out_channels': 32,
        'conv2d_kernel_size': 3,
        'conv2d_padding': 1,
        # maxpool2d 사이트 보면 kernel_size 반드시 정의해야 됨
        'maxpool2d_kernel_size': 2,
        'maxpool2d_stride': 2,
    },
    'layer_2': {
        'conv2d_in_channels': 32, # layer1의 out channel이 32였으니까
        'conv2d_out_channels': 64,
        'conv2d_kernel_size': 3,
        'conv2d_padding': 0,
        'maxpool2d_kernel_size': 2,
        'maxpool2d_stride': 1,

    },
    # fully connected layer
    'fc_1': {
        # input features -> 일단 임의의 값 '7011' 넣기 (에러 메시지 보고 고치기)
        # 나중에 model build 해 보면 multiply 안 된다는 에러 발생 : 7744로 변경
        'in_features': 7744,
        'out_features': 512
    },
    'fc_2': {
        'in_features': 512,   # fc_1의 out이 512였으니까
        'out_features': 128
    },
    'fc_3': {
        'in_features': 128,   # fc_2의 out이 128였으니까
        'out_features': 10    # 최종 class 수
    },
    'dropout_prob': 0.25,
}

_cnn_cfg = OmegaConf.create(_cnn_cfg_dict)
# print(_cnn_cfg)                                # {'layer_1': {}, 'layer_2': {}, 'fc_1': {}, 'fc_2': {}, 'fc_3': {}}
# print(_cnn_cfg.layer_1, _cnn_cfg['layer_1'])   # {} {}
print(OmegaConf.to_yaml(_cnn_cfg))               # to_yaml : 위 두 개 코드보다 좀 더 보기 좋게 프린트

# 결과 저장
# with open('cnn_test.ymal', 'w') as f:
#     OmegaConf.save(_cnn_cfg, f)

# 불러오기
# OmegaConf.load

class CNN(nn.Module):
    def __init__(self, cfg: DictConfig = _cnn_cfg):
        super().__init__()
        # convolutional layer와 maxpooling layer를 동시에 쓰려면 nn.Sequential로 같이 묶어줘야 함
        self.layer1 = nn.Sequential(
            # convolutional layer -> batch normalization -> relu -> maxpooling layer
            nn.Conv2d(
                in_channels=cfg.layer_1.conv2d_in_channels,
                out_channels=cfg.layer_1.conv2d_out_channels,
                kernel_size=cfg.layer_1.conv2d_kernel_size,
                padding=cfg.layer_1.conv2d_padding
            ),
            nn.BatchNorm2d(cfg.layer_1.conv2d_out_channels),  # num_features
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size=cfg.layer_1.maxpool2d_kernel_size,
                stride=cfg.layer_1.maxpool2d_stride
            )
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(
                in_channels=cfg.layer_2.conv2d_in_channels,
                out_channels=cfg.layer_2.conv2d_out_channels,
                kernel_size=cfg.layer_2.conv2d_kernel_size,
                padding=cfg.layer_2.conv2d_padding
            ),
            nn.BatchNorm2d(cfg.layer_2.conv2d_out_channels),  # num_features
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size=cfg.layer_2.maxpool2d_kernel_size,
                stride=cfg.layer_2.maxpool2d_stride
            )
        )

        self.fc1 = nn.Linear(
            in_features=cfg.fc_1.in_features,
            out_features=cfg.fc_1.out_features,
        )
        self.fc2 = nn.Linear(
            in_features=cfg.fc_2.in_features,
            out_features=cfg.fc_2.out_features,
        )
        self.fc3 = nn.Linear(
            in_features=cfg.fc_3.in_features,
            out_features=cfg.fc_3.out_features,
        )

        # conv, maxpool 모두 2d를 계속 사용했으니 dropout도 2d로 함
        self.dropout = nn.Dropout2d(cfg.dropout_prob)
    
    # output은 이미지 형태인데 fully connected는 2d 이미지가 아닌 걸로 가정하기 때문에 중간에 한 번 flatten 해줘야 함
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        # flatten
        out = out.view(out.size(0), -1)  # size(0) : batch size, -1 : 앞에 있는 size(batch)만 남기고 나머지 flatten 해라
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

CNN()

Efficient Net
- 현재 우리가 GPU 1개만 쓰고 있으니
  - 깊은 모델은 어렵고 efficientnet-b0, b1 정도 가능
  - pretrain 안 하고 이미 pretrained 된 모델로 사용

In [ ]:
_efficient_finetune_cfg_dict: dict = {
    "efficient_net_model_name": "efficientnet-b1",
    "num_classes": 10
}

_efficient_finetune_cfg_dict = OmegaConf.create(_efficient_finetune_cfg_dict)
print(OmegaConf.to_yaml(_efficient_finetune_cfg_dict))

class EfficientNetFinetune(nn.Module):
    def __init__(self, cfg: DictConfig = _efficient_finetune_cfg_dict):
        super().__init__()
        self.efficientnet = EfficientNet.from_pretrained(
            cfg.efficient_net_model_name,
            cfg.num_classes   # default : 1000개

        )
    
    def forward(self, x):
        out = self.efficientnet(x)
        return out

## Learning Rate Scheduler
- https://pytorch.org/docs/stable/optim.html
- https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.LambdaLR.html

In [ ]:
# Warmup Scheduler
class WarmupLR(optim.lr_scheduler.LambdaLR):

    def __init__(
        self,
        optimizer: optim.Optimizer,
        warmup_end_steps: int,
        last_epoch: int = -1,
        ):

        # warmup lr scheduler
        # warmup은 Adam Optimizer의 단점을 보완하기 위해 나옴 - Adam Optimizer와 같이 사용해보기
        def warmup_fn(step: int):
            if step < warmup_end_steps:
                # max(warmup_end_steps, 1) : 1 이상인 값으로 인지하도록
                return float(step) / float(max(warmup_end_steps, 1))
            return 1.0

        # warmup_fn : lr_lambda 값으로 입력
        super().__init__(optimizer, warmup_fn, last_epoch)

## 모델 선언 및 손실 함수, 최적화(optimizer) 정의, Tensorboard Logger 정의

- 아래 셀 값들 조절해서 다른 모델 만들어 볼 수 있음

### GPU setting

In [ ]:
# gpu = None  # 코드 에러 날 때 cpu는 잘 작동하는지 확인하기 위해 gpu=None으로 설정
gpu = 0   # gpu를 0번 쓰겠다는 의미

### define model
- input shape : 28*28
- hidden layer : 임의로 일단 128, 64로 정함 -> 수정 가능 (0.5배, 2배, 3배 등)
- output : label 0~9까지 있으니 10

In [ ]:
# model = MLP(28*28, 128, 64, 10)
# model = MLPWithDropout(28*28, 128, 64, 10, dropout_prob=0.3)
# model = CNN(cfg=_cnn_cfg)
model = EfficientNetFinetune(cfg=_efficient_finetune_cfg_dict)

# ★★ model을 GPU에 태우겠다는 의미
# 안 쓰면 아래와 같은 error 발생
# RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same
if gpu is not None:
    model.cuda(gpu)

model_name = type(model).__name__
print(model_name)

# define loss
loss_function = nn.CrossEntropyLoss()

### define optimizer
- 공식문서 : https://pytorch.org/docs/stable/optim.html
- open source : https://github.com/jettify/pytorch-optimizer
  - 유명하거나 많이 쓰이는 것을 모아 놓은 곳이라 다른 open source 대비 신뢰성↑

Adam vs SGD
- Adam이 tuning 등 손이 덜 가고 사용하기 편리
- SGD tuning을 굉장히 잘 하면 Adam보다 성능 좋을 수 있음
- 단, SGD는 Adam보다 학습 속도가 훨씬 느림
- learning rate 값 조절에 따른 효과 보기 위해서는 기본 optimizer인 SGD 사용하는 게 좋음

RAdam : Rectified Adam
- 뭘 써야할 지 잘 모르겠으면 RAdam 쓰는 게 제일 좋음

AdamP
- Adam과 거의 비슷하지만 Adam이 툭 튀는 부분을 잡아주는 optimizer = Adam보다 조금 나음

In [ ]:
# finetune에서는 보통 learing rate(1ㄷ-3)보다 조금 더 작은 값 쓰면 모델 결과 더 좋게 나오는 경우 많음
lr = 1e-3

# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = RAdam(model.parameters(), lr=lr)
# optimizer = AdamP(model.parameters(), lr=lr)
optimizer_name = type(optimizer).__name__

# define scheduler
scheduler = None
# scheduler = WarmupLR(optimizer, 1500)
scheduler_name = type(scheduler).__name__ if scheduler is not None else 'no'

max_epoch = 50

### define tensorboard logger
- .isoformat(timespec='seconds') : 초 단위까지만 시간 나타나게 하기

In [ ]:
# 현재 위치를 예전 모델들 결과물이 있는 폴더로 설정해 놓아서
# 모든 결과물이 같은 폴더 내에 있으면 tensorboard에서 모델 간 비교 가능
run_name = f"{datetime.now().isoformat(timespec='seconds')}-{model_name}-{optimizer_name}_optim_{lr}_lr_with_{scheduler_name}_scheduler"
run_dirname= 'dnn_tutorial-fashion-mnist-runs'
# log_dir = f'runs/{run_name}'
log_dir = os.path.join(drive_project_root, "runs", run_dirname, run_name)
writer = SummaryWriter(log_dir=log_dir)
log_interval = 100

### define wandb
- wandb 사용하기
  - 가입
  - browser here 링크 클릭하면 코드 나옴
  - 해당 코드 입력하면 로그인 됨
  - project page 링크 클릭
- wandb는 tensorboard에서 보여주지 않는 것들 보여줌
  - hardware : GPU 쓰다 보면 처음 보는 에러 발생해서 죽는 경우 있는데 그때 보통 hardware 에러 - wandb를 통해 문제 파악 가능

In [ ]:
project_name = 'fastcampus_fashion_mnist_tutorials'
run_tags = [project_name]
wandb.init(
    project=project_name,
    name=run_name,
    tags=run_tags,
    config={"lr": lr, "model_name": model_name, "optimizer_name": optimizer_name, "scheduler_name": scheduler_name},
    reinit=True,
)

# 모델 그래프 보기
wandb.watch(model)

### set save model path
- 이렇게 서로 다른 파일에 모델 결과들을 저장해 놓아야 여러 모델을 돌릴 때 비교 가능

In [ ]:
log_model_path = os.path.join(log_dir, 'models')
os.makedirs(log_model_path, exist_ok=True)

### scheduler 확인

- linear하게 증가하는 것 볼 수 있음
- step 0 : 6.666666666666667e-07 -> 0.001/1500
  - 0.001 : learning rate 값
  - 1500 : scheduler = WarmupLR(optimizer, 1500)에서 사용된 값
- optimizer에 warmup_fn에서 return한 '1.0'이 곱해짐

In [ ]:
# for i in range(250):
#     print("step", i)
#     optimizer.step()
#     scheduler.step()
#     print(scheduler.get_last_lr())

## early stopping callback object class 정의
- with some modification, source is from [here](https://github.com/bjarten/early-stopping-pytorch)

- patience (int): How long to wait after last time validation loss improved (default 7)
- verbose (bool): If True, print a message for each validation loss improvement (default False)
- delta (float) : Minimum change in the monitored quantity to qualify as an improvement (default 0)
- path (str): Path for the checkpoint to be saved to (default 'checkpoint.ckpt') - epoch가 돌 때마다 모델 저장
- trace_func (function): trace print function (default print)            

In [ ]:
class EarlyStopping:
    # Early stops the training if validation loss doesn't improve after a given atience
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.ckpt', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            # score가 최대가 되었을 때 = loss가 최소가 되었을 때 저장
            self.save_checkpoint(val_loss, model)
        # best_score보다 score 값이 적을 때 = 모델 성능이 더이상 개선되지 않을 때
        # 이론과 달리 실전에서는 model이 계속 개선되다가 특정 구간에서 다시 계속 증가하지 않고
        # 약간 울렁울렁 올라갔다 내려갔다 변동하는 게 있음
        # -> 그래서 patience 사용하는 것 (n번정도까지 기다려 봄)
        #    그래도 성능 개선이 없으면 early stop 'True'
        #    위에서 model save_checkpoint를 해줬기 때문에
        #    좀 더 학습하더라도 저장된 최종 모델을 사용하면 되니 안전함
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
    
    def save_checkpoint(self, val_loss, model):
        # Svaes model when validation loss decrease
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...')
        
        filename = self.path.split('/')[-1]
        save_dir = os.path.dirname(self.path)
        torch.save(model, os.path.join(save_dir, f'val_loss-{val_loss}-{filename}'))
        self.val_loss_min = val_loss

In [ ]:
%load_ext tensorboard
# runs라는 폴더를 만들고 그 안에 logging을 쌓을 예정
# %tensorboard --logdir runs/

# 현재 위치를 예전 모델들 결과물이 있는 폴더로 설정해 놓아서
# 모든 결과물이 같은 폴더 내에 있으면 tensorboard에서 모델 간 비교 가능
# 경로 지정 : terminal이기 때문에 '#'를 '#'으로 인식하려면 앞에 '\' 입력 필요
%tensorboard --logdir /content/drive/MyDrive/\#fastcampus/runs/dnn-tutorial-fashion-mnist-runs/

# define EarlyStopping
early_stopper = EarlyStopping(
    patience=3, verbose=True, path=os.path.join(log_model_path, 'model.ckpt')
)

# Do train with validation
train_step = 0
for epoch in range(1, max_epoch+1):
    
    # valid step
    # ★TIP : train step보다 먼저 해 보면 랜덤 input에 대해 얼마나 막장인지 미리 결과 확인해 볼 수 있으니 좋음
    # 단, validation data에 대해서는 optimizer를 하면 안 됨 -> cheating이니까
    # pytorch에서는 'torch.no_grad()'만 해도 optimizer 안 한 것으로 바꿔줌
    with torch.no_grad():
        val_loss = 0.0
        val_corrects = 0
        model.eval()   # 지금은 model evaluation 하는 time이라는 것 알려줌 (model.train()과 세트)

        # enumerate 안에 tqdm 해 주면 결과를 더 예쁘게 볼 수 있음
        # https://tqdm.github.io/docs/tqdm/
        # - position : Specify the line offset to print this bar (starting from 0) -> Useful to manage multiple bars at once
        #              ipynb은 자꾸 밑으로 내려가는데(?) 'position=0'으로 해야 가만히 있음 
        # - leave=True : keeps all traces of the progressbar upon termination of iteration
        # - desc : 진행바 앞에 text 출력
        for val_batch_idx, (val_images, val_labels) in enumerate(
            tqdm(val_dataloader, position=0, leave=True, desc="validation")
            ):

            # ★ cpu가 아닌 gpu 사용할 때는 이 코드 넣어줘야 함
            if gpu is not None:
                val_images = val_images.cuda(gpu, non_blocking=True)
                val_labels = val_labels.cuda(gpu, non_blocking=True)

            # forward
            val_outputs = model(val_images)
            _, val_preds = torch.max(val_outputs, 1)

            # loss & acc
            # val_outputs.shape[0] : batch size
            val_loss += loss_function(val_outputs, val_labels) / val_outputs.shape[0]
            val_corrects += torch.sum(val_preds == val_labels.data) / val_outputs.shape[0]
            
    # valid step logging
    val_epoch_loss = val_loss / len(val_dataloader)
    val_epoch_acc = val_corrects / len(val_dataloader)

    print(
        f"{epoch} epoch, {train_step} step: val_loss: {val_epoch_loss}, val_acc: {val_epoch_acc}"
    )

    # tensorboard log : tensorboard에 write
    # val_step 말고 train_step으로 해야 서로 같은 시점에서 비교하기 좋음
    writer.add_scalar('Loss/val', val_epoch_loss, train_step)
    writer.add_scalar('Acc/val', val_epoch_acc, train_step)
    writer.add_images('Images/val', val_images, train_step)

    # wandb log
    wandb.log({
        'Loss/val': val_epoch_loss,
        'Acc/val': val_epoch_acc,
        # image는 그냥 쓸 수 없고 wandb.Image 써줘야 함
        'Images/val': wandb.Image(val_images),
        # histogram
        # prediction 하고 있는 게 한 가지 class로 overfitting 되고 있지 않은 지 확인 가능
        # detach() : gpu를 쓰고 있다면 detach
        # ★ .cpu() : gpu 쓰던 중이었으면 detach 하고 cpu로 보내주는 것까지 해야 numpy로 변환 가능
        'Outputs/val': wandb.Histogram(val_outputs.detach().cpu().numpy()),
        'Preds/val': wandb.Histogram(val_preds.detach().cpu().numpy()),
        'Labels/val': wandb.Histogram(val_labels.data.detach().cpu().numpy())
    }, step=train_step)

    # check model early stopping print & save model if model reached the best perormance
    early_stopper(val_epoch_loss, model)   # call 부분
    if early_stopper.early_stop:           # early_stop = True면 학습 중단
        break

    # ===============================================================

    # train step
    current_loss = 0
    current_corrects = 0
    model.train()     # 지금은 model training 하는 time이라는 것 알려줌 (model.eval()과 세트)

    for batch_idx, (images, labels) in enumerate(
        tqdm(train_dataloader, position=0, leave=True, desc='training')
        ):

        # ★ cpu가 아닌 gpu 사용할 때는 이 코드 넣어줘야 함
        if gpu is not None:
            images =  images.cuda(gpu)
            labels = labels.cuda(gpu)

        current_loss = 0.0
        current_corrects = 0  # 몇 개나 맞췄는지
        
        # Forward ===================================================
        # get predictions
        outputs = model(images)
        _, preds = torch.max(outputs, 1)  # 1 : dimension
        # print(outputs)
        # print(preds)

        # get loss
        # Cross Entropy : loss_function(input, target)
        loss = loss_function(outputs, labels)
        
        # Backpropagation ===========================================
        # optimizer 초기화 (zero화) : garbage 값이 있을 수도 있으니 처리해주기
        optimizer.zero_grad()

        # Perform backward pass
        # Chain rule 자동 계산해 줌
        loss.backward()

        # Perform Optimization
        optimizer.step()

        # Perform LR scheduler work
        if scheduler is not None:
            scheduler.step()  # 이거 안 하면 learning rate 제대로 안 나옴

        current_loss += loss.item()
        current_corrects += torch.sum(preds == labels.data)

        # 일정 이상 돌면 = 100번(log_interval)마다 정확도 평균 계산
        if train_step % log_interval == 0:
            train_loss = current_loss / log_interval
            train_acc = current_corrects / log_interval

            print(
                f'{train_step}: train_loss: {train_loss}, train_acc: {train_acc}'
            )

            # current learning rate
            # scheduler를 사용하지 않는 경우 대비
            # 아래 코드 없이 돌리면 Error 발생 : 'NoneType' object has no attribute 'get_last_lr'
            cur_lr = optimizer.param_groups[0]['lr'] if scheduler is None else scheduler.get_last_lr()[0]

            # tensorboard log : tensorboard에 write
            writer.add_scalar('Loss/train', train_loss, train_step)
            writer.add_scalar('Acc/train', train_acc, train_step)
            writer.add_images('Images/train', images, train_step)
            # scheduler : list형태 return -> [0] 값 가져오기
            writer.add_scalar("Learning Rate", cur_lr, train_step)
            writer.add_graph(model, images)

            # wandb log
            wandb.log({
                'Loss/train': train_loss,
                'Acc/train': train_acc,
                'Images/train': wandb.Image(images),
                # ★ .cpu() : gpu 쓰던 중이었으면 detach 하고 cpu로 보내주는 것까지 해야 numpy로 변환 가능
                'Outputs/train': wandb.Histogram(outputs.detach().cpu().numpy()),
                'Preds/train': wandb.Histogram(preds.detach().cpu().numpy()),
                'Labels/train': wandb.Histogram(labels.data.detach().cpu().numpy()),
                'Learning Rate': cur_lr,
            }, step=train_step)

            # loss 초기화
            current_loss = 0
            current_corrects= 0

        train_step += 1
        # break
    # break

## save model

- ckpt : checkpoint

In [ ]:
# torch.save(model, os.path.join(log_model_path, "model.ckpt"))

In [ ]:
log_model_path

load model

- eval(expression) : expression을 문자열로 받아 실행

In [ ]:
# loaded_model = torch.load(os.path.join(log_model_path, "model.ckpt"))
loaded_model = torch.load(os.path.join(log_model_path, 'val_loss-0.029985815286636353-model.ckpt'))
loaded_model.eval()
loaded_model.cpu()   # gpu 사용했다면 cpu로 바꿔주기 -> 아래 test 코드는 cpu용으로 되어 있으니 error 발생
print(loaded_model)  # 잘 load 되었는지 확인

## softmax

In [ ]:
def softmax(x, axis=0):
    'numpy softmax'
    max = np.max(x, axis=axis, keepdims=True)
    e_x = np.exp(x - max)
    sum = np.sum(e_x, axis=axis, keepdims=True)
    f_x = e_x / sum
    return f_x

## test model

torch.tensor
- It has an additional "computational graph" layer
- To convert a `torch.tensor` to `np.ndarray`, you must explicitly remove the computational graph of the tensor using the ★ `detach()` command

★ 분석을 할 때는 마지막에 다 numpy로 바꿔줘야 scikit learn 등 여러 가지 tool에 쉽고 유연하게 적용이 가능하기 때문에 아래와 같이 대부분 마지막에는 numpy로 바꿔주고 끝냄

In [ ]:
test_batch_size = 100
test_dataset = FashionMNIST(data_root, download=True, train=False, transform=transforms.ToTensor())
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=1)

test_labels_list = []
test_preds_list = []
test_outputs_list = []

for i, (test_images, test_labels) in enumerate(tqdm(test_dataloader, position=0, leave=True, desc='testing')):
    # forward
    test_outputs = loaded_model(test_images)
    _, test_preds = torch.max(test_outputs, 1)

    final_outs = softmax(test_outputs.detach().numpy(), axis=1)
    test_outputs_list.extend(final_outs)
    test_preds_list.extend(test_preds.detach().numpy())
    test_labels_list.extend(test_labels.detach().numpy())

# accuracy를 구하기 위해 numpy array로 변경
test_preds_list = np.array(test_preds_list)
test_labels_list = np.array(test_labels_list)

print(f'\nacc: {np.mean(test_preds_list == test_labels_list)*100}%')

## ROC curve

- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

fpr = {}
tpr = {}
thresh = {}  # threshold
n_class = 10

for i in range(n_class):
    # [:, i] : batch size는 유지하고 각 class마다 확인
    fpr[i], tpr[i], thresh[i] = roc_curve(test_labels_list, np.array(test_outputs_list)[:, i], pos_label=i)

전체 test case에 대한 값

In [ ]:
print(fpr)

In [ ]:
for i in range(n_class):
    plt.plot(fpr[i], tpr[i], linestyle='--', label=f'Class {i} vs Rest')
plt.title('Multi-class ROC Curve')
plt.xlabel('False Position Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='best')
plt.show()

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html

- ovo : Stands for One-vs-one. Computes the average AUC of all possible pairwise combinations of classes.
- macro : Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

In [ ]:
print('auc_score ', roc_auc_score(test_labels_list, test_outputs_list, multi_class='ovo', average='macro'))